In [23]:
%matplotlib qt
import lmfit
import numpy as np
from bluesky import RunEngine
from bluesky.plans import scan, rel_scan, list_scan, rel_list_scan, log_scan
from bluesky.plan_stubs import mv
from bluesky.callbacks import LiveFitPlot, LiveFit, LivePlot
from bluesky.callbacks.fitting import PeakStats
from bluesky.callbacks.mpl_plotting import plot_peak_stats
from bluesky.callbacks.best_effort import BestEffortCallback
from ophyd.sim import det, motor, noisy_det
from matplotlib.pyplot import ion, subplots
from bluesky.utils import install_nb_kicker
install_nb_kicker()

ion()

In [24]:
RE = RunEngine()

bec = BestEffortCallback()
RE.subscribe(bec)

0

## Scan

In [3]:
RE(
    scan([det], motor, -1, 1, num=20)
)



Transient Scan ID: 1     Time: 2024-08-28 11:22:35
Persistent Unique Scan ID: '9247b783-1b56-4633-afc7-25568043b0f2'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |      motor |        det |
+-----------+------------+------------+------------+
|         1 | 11:22:35.4 |     -1.000 |      0.607 |
|         2 | 11:22:35.4 |     -0.895 |      0.670 |
|         3 | 11:22:35.8 |     -0.789 |      0.732 |
|         4 | 11:22:35.8 |     -0.684 |      0.791 |
|         5 | 11:22:35.9 |     -0.579 |      0.846 |
|         6 | 11:22:35.9 |     -0.474 |      0.894 |
|         7 | 11:22:36.0 |     -0.368 |      0.934 |
|         8 | 11:22:36.0 |     -0.263 |      0.966 |
|         9 | 11:22:36.1 |     -0.158 |      0.988 |
|        10 | 11:22:36.2 |     -0.053 |      0.999 |
|        11 | 11:22:36.2 |      0.053 |      0.999 |
|        12 | 11:22:36.3 |      0.158 |      0.988 |
|        13 | 11:22:36.3 |      0.263 |      0.966 |
|        14

('9247b783-1b56-4633-afc7-25568043b0f2',)

In [4]:
RE(
    scan([det], motor, -5, 5, num=20)
)



Transient Scan ID: 2     Time: 2024-08-28 11:22:38
Persistent Unique Scan ID: '8f948d7a-33bb-4994-a215-92242f09ca93'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |      motor |        det |
+-----------+------------+------------+------------+
|         1 | 11:22:38.5 |     -5.000 |      0.000 |
|         2 | 11:22:38.5 |     -4.474 |      0.000 |
|         3 | 11:22:38.7 |     -3.947 |      0.000 |
|         4 | 11:22:38.8 |     -3.421 |      0.003 |
|         5 | 11:22:38.9 |     -2.895 |      0.015 |
|         6 | 11:22:39.0 |     -2.368 |      0.061 |
|         7 | 11:22:39.1 |     -1.842 |      0.183 |
|         8 | 11:22:39.3 |     -1.316 |      0.421 |
|         9 | 11:22:39.4 |     -0.789 |      0.732 |
|        10 | 11:22:39.5 |     -0.263 |      0.966 |
|        11 | 11:22:39.6 |      0.263 |      0.966 |
|        12 | 11:22:39.7 |      0.789 |      0.732 |
|        13 | 11:22:39.9 |      1.316 |      0.421 |
|        14

('8f948d7a-33bb-4994-a215-92242f09ca93',)

In [5]:
RE(
    scan([det], motor, -5, 5, num=100)
)



Transient Scan ID: 3     Time: 2024-08-28 11:22:41
Persistent Unique Scan ID: '34ecd075-c08a-43cf-ad3f-aff05990f923'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |      motor |        det |
+-----------+------------+------------+------------+
|         1 | 11:22:41.3 |     -5.000 |      0.000 |
|         2 | 11:22:41.3 |     -4.899 |      0.000 |
|         3 | 11:22:41.4 |     -4.798 |      0.000 |
|         4 | 11:22:41.5 |     -4.697 |      0.000 |
|         5 | 11:22:41.6 |     -4.596 |      0.000 |
|         6 | 11:22:41.8 |     -4.495 |      0.000 |
|         7 | 11:22:41.8 |     -4.394 |      0.000 |
|         8 | 11:22:41.9 |     -4.293 |      0.000 |
|         9 | 11:22:42.0 |     -4.192 |      0.000 |
|        10 | 11:22:42.1 |     -4.091 |      0.000 |
|        11 | 11:22:42.1 |     -3.990 |      0.000 |
|        12 | 11:22:42.2 |     -3.889 |      0.001 |
|        13 | 11:22:42.2 |     -3.788 |      0.001 |
|        14

('34ecd075-c08a-43cf-ad3f-aff05990f923',)

## Plot fitting

In [26]:
fig, ax = subplots()

maximum = 2
sigma = 3

def gaussian(x, max, sigma, x0):
    return max*np.exp(-(x - x0)**2/(2 * sigma**2))

model = lmfit.Model(gaussian)
init_guess = {
    'max': maximum,
    'sigma': lmfit.Parameter('sigma', sigma, min=0),
    'x0': 0
}
live_fit = LiveFit(model, 'noisy_det', {'x': 'motor'}, init_guess)
live_fit_plot = LiveFitPlot(live_fit, color='r', ax=ax)

In [28]:
live_plot = LivePlot('noisy_det', 'motor', marker='x', linestyle='none', ax=ax)

bec.disable_plots()
RE(
    scan([noisy_det], motor, -5, 5, num=40),
    [live_fit_plot, live_plot]
)



Transient Scan ID: 2     Time: 2024-08-28 13:48:26
Persistent Unique Scan ID: '0aec6916-2bd2-48c3-8800-527e37ad72a3'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |      motor |  noisy_det |
+-----------+------------+------------+------------+
|         1 | 13:48:26.4 |     -5.000 |      0.068 |
|         2 | 13:48:26.5 |     -4.744 |     -0.015 |
|         3 | 13:48:26.6 |     -4.487 |      0.038 |
|         4 | 13:48:26.6 |     -4.231 |      0.046 |
|         5 | 13:48:26.7 |     -3.974 |     -0.002 |
|         6 | 13:48:26.7 |     -3.718 |      0.077 |
|         7 | 13:48:26.7 |     -3.462 |     -0.084 |
|         8 | 13:48:26.8 |     -3.205 |     -0.083 |
|         9 | 13:48:27.0 |     -2.949 |     -0.058 |
|        10 | 13:48:27.2 |     -2.692 |     -0.052 |
|        11 | 13:48:27.3 |     -2.436 |      0.141 |
|        12 | 13:48:27.5 |     -2.179 |      0.117 |
|        13 | 13:48:27.6 |     -1.923 |      0.241 |
|        14

('0aec6916-2bd2-48c3-8800-527e37ad72a3',)

### Peak Stats

* cen: center of mass
* com: mid-point between half-max points on each side of the peak
* max: location of y maximum
* min: location of y minimum

In [30]:
ps = PeakStats('motor', 'noisy_det', calc_derivative_and_stats=True)
RE(
    scan([noisy_det], motor, -1, 4.5, num=30), ps
)

plot_peak_stats(ps)
bec.enable_plots()



Transient Scan ID: 4     Time: 2024-08-28 13:49:00
Persistent Unique Scan ID: '83658d58-8c2a-49ee-bf6b-47cf6dcddbd7'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |      motor |  noisy_det |
+-----------+------------+------------+------------+
|         1 | 13:49:00.6 |     -1.000 |      0.518 |
|         2 | 13:49:00.6 |     -0.810 |      0.642 |
|         3 | 13:49:00.7 |     -0.621 |      0.907 |
|         4 | 13:49:00.8 |     -0.431 |      0.969 |
|         5 | 13:49:00.8 |     -0.241 |      1.069 |
|         6 | 13:49:00.9 |     -0.052 |      1.067 |
|         7 | 13:49:00.9 |      0.138 |      1.088 |
|         8 | 13:49:00.9 |      0.328 |      0.900 |
|         9 | 13:49:01.0 |      0.517 |      0.804 |
|        10 | 13:49:01.0 |      0.707 |      0.689 |
|        11 | 13:49:01.0 |      0.897 |      0.726 |
|        12 | 13:49:01.0 |      1.086 |      0.480 |
|        13 | 13:49:01.1 |      1.276 |      0.411 |
|        14

## Relative Scan

In [31]:
RE(mv(motor, 3))
RE(
    rel_scan([det], motor, -1, 1, num=50)
)



Transient Scan ID: 5     Time: 2024-08-28 14:08:40
Persistent Unique Scan ID: '849d0382-2f99-44a9-a0ad-a2372be1e961'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |      motor |        det |
+-----------+------------+------------+------------+
|         1 | 14:08:40.5 |      2.000 |      0.135 |
|         2 | 14:08:41.0 |      2.041 |      0.125 |
|         3 | 14:08:41.3 |      2.082 |      0.115 |
|         4 | 14:08:41.3 |      2.122 |      0.105 |
|         5 | 14:08:41.4 |      2.163 |      0.096 |
|         6 | 14:08:41.5 |      2.204 |      0.088 |
|         7 | 14:08:41.6 |      2.245 |      0.080 |
|         8 | 14:08:41.7 |      2.286 |      0.073 |
|         9 | 14:08:41.7 |      2.327 |      0.067 |
|        10 | 14:08:41.8 |      2.367 |      0.061 |
|        11 | 14:08:41.8 |      2.408 |      0.055 |
|        12 | 14:08:41.9 |      2.449 |      0.050 |
|        13 | 14:08:42.0 |      2.490 |      0.045 |
|        14

('849d0382-2f99-44a9-a0ad-a2372be1e961',)

## List Scan

In [8]:
RE(
    list_scan([det], motor, [-1.4, -1.3, -0.8, -0.7, -0.2, -0.1])
)



Transient Scan ID: 6     Time: 2024-08-28 11:22:57
Persistent Unique Scan ID: '975f51d7-557b-4ac5-859c-f91c5d0e1c95'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |      motor |        det |
+-----------+------------+------------+------------+
|         1 | 11:22:57.1 |     -1.400 |      0.375 |
|         2 | 11:22:57.1 |     -1.300 |      0.430 |
|         3 | 11:22:57.1 |     -0.800 |      0.726 |
|         4 | 11:22:57.2 |     -0.700 |      0.783 |
|         5 | 11:22:57.2 |     -0.200 |      0.980 |
|         6 | 11:22:57.2 |     -0.100 |      0.995 |
+-----------+------------+------------+------------+
generator list_scan ['975f51d7'] (scan num: 6)





('975f51d7-557b-4ac5-859c-f91c5d0e1c95',)

## Relative List Scan

In [17]:
RE(mv(motor, 3))
RE(
    rel_list_scan([det], motor, [-1.4, -1.3, -0.8, -0.7, -0.2, -0.1])
)



Transient Scan ID: 13     Time: 2024-08-28 13:24:37
Persistent Unique Scan ID: '52a70295-ec0a-41c7-96d7-03839f6130b6'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |      motor |        det |
+-----------+------------+------------+------------+
|         1 | 13:24:37.8 |      1.600 |      0.278 |
|         2 | 13:24:37.9 |      1.700 |      0.236 |
|         3 | 13:24:37.9 |      2.200 |      0.089 |
|         4 | 13:24:37.9 |      2.300 |      0.071 |
|         5 | 13:24:38.0 |      2.800 |      0.020 |
|         6 | 13:24:38.0 |      2.900 |      0.015 |
+-----------+------------+------------+------------+
generator rel_list_scan ['52a70295'] (scan num: 13)





('52a70295-ec0a-41c7-96d7-03839f6130b6',)

## Log Scan

In [10]:
RE(
    log_scan([det], motor, -1, 1, num=25)
)



Transient Scan ID: 8     Time: 2024-08-28 11:22:58
Persistent Unique Scan ID: 'c319a2a8-11a6-4679-8d27-08446b7e0e8a'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |      motor |        det |
+-----------+------------+------------+------------+
|         1 | 11:22:58.1 |      0.100 |      0.995 |
|         2 | 11:22:58.2 |      0.121 |      0.993 |
|         3 | 11:22:58.2 |      0.147 |      0.989 |
|         4 | 11:22:58.2 |      0.178 |      0.984 |
|         5 | 11:22:58.2 |      0.215 |      0.977 |
|         6 | 11:22:58.3 |      0.261 |      0.967 |
|         7 | 11:22:58.3 |      0.316 |      0.951 |
|         8 | 11:22:58.3 |      0.383 |      0.929 |
|         9 | 11:22:58.3 |      0.464 |      0.898 |
|        10 | 11:22:58.4 |      0.562 |      0.854 |
|        11 | 11:22:58.4 |      0.681 |      0.793 |
|        12 | 11:22:58.5 |      0.825 |      0.711 |
|        13 | 11:22:58.5 |      1.000 |      0.607 |
|        14

('c319a2a8-11a6-4679-8d27-08446b7e0e8a',)